In [1]:
import pandas as pd
import sqlite3
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

In [2]:
excel_file = 'Docs/Python Batch 01.xlsx'
db_file = 'student_details.db'

conn = sqlite3.connect(db_file)
df = pd.read_excel(excel_file)
df.to_sql('python_batch_1', conn, if_exists='replace', index=False)

# conn.close()

214

In [3]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print(f"Found {len(tables)} tables in the database")

Found 1 tables in the database


In [4]:
# For each table, get column count and row count
for table in tables:
    table_name = table[0]
    
    # Get column information
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    
    # Load table into DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    
    # Print information about the table
    print(f"\nTable: {table_name}")
    print(f"Number of columns: {len(columns)}")
    print(f"Number of rows: {len(df)}")
    
    # Print column names if needed
    print("Column names:")
    for i, col in enumerate(df.columns):
        print(f"  {i+1}. {col}")


Table: python_batch_1
Number of columns: 9
Number of rows: 214
Column names:
  1.  Index no.
  2. Email Address
  3. Full name
  4. WhatsApp mobile number
  5. Current State
  6. University name
  7. Year
  8. Have any knowledge of programming
  9. Any experience in Python


In [5]:
df.head()

,Index no.,Email Address,Full name,WhatsApp mobile number,Current State,University name,Year,Have any knowledge of programming,Any experience in Python
0,2101001,thushandamintha.satd24@gmail.com,Thushan Damintha Sooriyaarachchi,0712821572,Undergraduate,University of Moratuwa,1st year,Yes,Yes
1,2101002,weeratharava@gmail.com,Thushalya Tharindu Weerasuriya,0754365392,After A/L,University of Moratuwa,1st year,Yes,No
2,2101003,gawrangarajapaksha@gmail.com,gawranga rajapaksha,0772564403,Before A/L,None,None,No,No
3,2101004,thushandamintha.satd24@gmail.com,Shashini rangika,0714913008,Before A/L,None,None,No,No
4,2101005,thisaruwan05@gmail.com,chalaka thisaruwan,0785933615,After A/L,Moratuwa,Waiting,No,No


In [6]:
from langchain.docstore.document import Document
# Read the Excel file
df = pd.read_excel('Docs/Python Batch 01.xlsx')

# Convert DataFrame to Document objects
documents = []
for index, row in df.iterrows():
    # Concatenate all columns into a single text field
    text = ' '.join([str(val) for val in row.values if pd.notna(val)])
    # Create a Document object
    doc = Document(page_content=text, metadata={"row": index})
    documents.append(doc)

# Split the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

embeddings = OllamaEmbeddings(model="nomic-embed-text")
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="local-rag"
)

In [7]:
llm = ChatOllama(model="llama3.2")

In [8]:
column_names = len(columns)
column_data_types = [column[2] for column in columns]

In [ ]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["column_names", "column_data_types", "context", "question"],
    template="""
    You are an expert SQL query generator. Based on the table schema and sample data provided, 
    create a SQL query to accomplish the task.

    TABLE SCHEMA:

    Column names list: {column_names}
    Table columns data types: {column_data_types}

    Generate a valid SQLite query to accomplish the task. Return your response ONLY as a JSON object with the following structure:
    {{
        "query number": "query 1",
        "SQL_Query": "Query"
    }}

    Original question: {question}

    Do not include any other text in your response - only the JSON object.
    """
)

In [13]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

template = """Answer the question based ONLY on the following context:
    {context}
    Question: {question}
    """

In [15]:
from langchain.chains import RetrievalQA

# chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | QUERY_PROMPT
#     | llm
#     | StrOutputParser()
# )

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_db.as_retriever(),
    chain_type_kwargs={
        "prompt": QUERY_PROMPT,
        # "additional_variables": {
        #     "column_names": column_names,
        #     "column_data_types": column_data_types
        # }
    }
)

ValidationError: 1 validation error for StuffDocumentsChain
  Value error, document_variable_name context was not found in llm_chain input_variables: ['column_data_types', 'column_names'] [type=value_error, input_value={'llm_chain': LLMChain(ve...None, 'callbacks': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [ ]:
responce = chain.invoke("what is university most attented students course in python_batch_1 table?")

KeyError: "Input to PromptTemplate is missing variables {'column_names', 'column_data_types'}.  Expected: ['column_data_types', 'column_names'] Received: ['question']\nNote: if you intended {column_names} to be part of the string and not a variable, please escape it with double curly braces like: '{{column_names}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "